In [1]:
import OpenDartReader
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os

In [2]:
url = 'https://docs.google.com/spreadsheets/d/1IunixzkpWvvNsIUJKN9JDZXRVWxVGFMpLBX-ht6tTVE/htmlview#'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
body = soup.find('tbody')
first_row = body.find_all('tr')[3]
first_row_data = first_row.find_all('td')
print(first_row_data[1].text, first_row_data[2].text, first_row_data[3].text) # 업종, 종목코드, 종목명

코스피 095570 AJ네트웍스


In [4]:
# save to pandas dataframe
pd_data = []
rows = body.find_all('tr')
for row in rows:
    data = row.find_all('td')
    if len(data) > 0 and data[1].text == '코스피':
        pd_data.append([data[1].text, data[2].text, data[3].text])

df_names = pd.DataFrame(pd_data, columns=['업종', '종목코드', '종목명'])
df_names.head()

,업종,종목코드,종목명
0,코스피,095570,AJ네트웍스
1,코스피,068400,AJ렌터카
2,코스피,006840,AK홀딩스
3,코스피,027410,BGF리테일
4,코스피,138930,BNK금융지주


In [5]:
api_key = os.getenv('API_KEY')

dart = OpenDartReader(api_key)

In [6]:
df_names[df_names['종목명'] == '한국패러랠']

,업종,종목코드,종목명
665,코스피,168490,한국패러랠


In [7]:
df = dart.finstate('삼성전자', 2022)
df = df[df["fs_div"] == "CFS"]
corpfile = df[['account_nm','thstrm_amount','frmtrm_amount','bfefrmtrm_amount']].set_index('account_nm')
profit = corpfile.loc['당기순이익'].tolist()
capital = corpfile.loc['자본총계'].tolist()
for value in profit:
    try:
        clean_value = int(value.replace(',', ''))
        profit = [clean_value if x == value else x for x in profit]
    except ValueError as e:
        print(f"Error: {e}")
for value in capital:
    try:
        clean_value = int(value.replace(',', ''))
        capital = [clean_value if x == value else x for x in capital]
    except ValueError as e:
        print(f"Error: {e}")
roe_list = []
for a in range(3):
     print(int(profit[a])/int(capital[a])*100)

15.688270366610475
13.08870417553489
9.56985753432632


In [13]:
print(corpfile)

                  thstrm_amount        frmtrm_amount     bfefrmtrm_amount
account_nm                                                               
유동자산        218,470,581,000,000  218,163,185,000,000  198,215,579,000,000
비유동자산       229,953,926,000,000  208,457,973,000,000  180,020,139,000,000
자산총계        448,424,507,000,000  426,621,158,000,000  378,235,718,000,000
유동부채         78,344,852,000,000   88,117,133,000,000   75,604,351,000,000
비유동부채        15,330,051,000,000   33,604,094,000,000   26,683,351,000,000
부채총계         93,674,903,000,000  121,721,227,000,000  102,287,702,000,000
자본금             897,514,000,000      897,514,000,000      897,514,000,000
이익잉여금       337,946,407,000,000  293,064,763,000,000  271,068,211,000,000
자본총계        354,749,604,000,000  304,899,931,000,000  275,948,016,000,000
매출액         302,231,360,000,000  279,604,799,000,000  236,806,988,000,000
영업이익         43,376,630,000,000   51,633,856,000,000   35,993,876,000,000
법인세차감전 순이익   46,440,474,000,000   53,3

In [8]:
def get_roe(stock_code):
    df = dart.finstate(stock_code, 2023)
    df = df[df["fs_div"] == "CFS"]
    corpfile = df[['account_nm','thstrm_amount','frmtrm_amount','bfefrmtrm_amount']].set_index('account_nm')
    profit = corpfile.loc['당기순이익'].tolist()
    capital = corpfile.loc['자본총계'].tolist()
    for value in profit:
        try:
            clean_value = int(value.replace(',', ''))
            profit = [clean_value if x == value else x for x in profit]
        except ValueError as e:
            print(f"Error: {e}")
    for value in capital:
        try:
            clean_value = int(value.replace(',', ''))
            capital = [clean_value if x == value else x for x in capital]
        except ValueError as e:
            print(f"Error: {e}")
    roe_list = []
    for a in range(3):
        roe_list.append(int(profit[a])/int(capital[a])*100)
    return roe_list[:,:,-1]

In [9]:
url = 'http://data-dbg.krx.co.kr/svc/sample/apis/sto/stk_bydd_trd' + '?basDd=20200102'
headers = { 'AUTH_KEY': '74D1B99DFBF345BBA3FB4476510A4BED4C78D13A'}
response = requests.get(url, headers=headers)
print(response.json())

{'OutBlock_1': [{'BAS_DD': '20200102', 'ISU_CD': '088980', 'ISU_NM': '맥쿼리인프라', 'MKT_NM': 'KOSPI', 'SECT_TP_NM': '-', 'TDD_CLSPRC': '11650', 'CMPPREVDD_PRC': '50', 'FLUC_RT': '0.43', 'TDD_OPNPRC': '11600', 'TDD_HGPRC': '11700', 'TDD_LWPRC': '11550', 'ACC_TRDVOL': '941567', 'ACC_TRDVAL': '10946554250', 'MKTCAP': '4066366514400', 'LIST_SHRS': '349044336'}, {'BAS_DD': '20200102', 'ISU_CD': '094800', 'ISU_NM': '맵스리얼티1', 'MKT_NM': 'KOSPI', 'SECT_TP_NM': '-', 'TDD_CLSPRC': '4890', 'CMPPREVDD_PRC': '-55', 'FLUC_RT': '-1.11', 'TDD_OPNPRC': '4985', 'TDD_HGPRC': '4985', 'TDD_LWPRC': '4890', 'ACC_TRDVOL': '113106', 'ACC_TRDVAL': '555882210', 'MKTCAP': '453959878590', 'LIST_SHRS': '92834331'}, {'BAS_DD': '20200102', 'ISU_CD': '204210', 'ISU_NM': '모두투어리츠', 'MKT_NM': 'KOSPI', 'SECT_TP_NM': '-', 'TDD_CLSPRC': '3120', 'CMPPREVDD_PRC': '-10', 'FLUC_RT': '-0.32', 'TDD_OPNPRC': '3165', 'TDD_HGPRC': '3180', 'TDD_LWPRC': '3090', 'ACC_TRDVOL': '3459', 'ACC_TRDVAL': '10698850', 'MKTCAP': '24419662800', 'LIST_

In [10]:
url = 'http://data-dbg.krx.co.kr/svc/sample/apis/sto/stk_bydd_trd' + '?basDd=20200102'
headers = { 'AUTH_KEY': '74D1B99DFBF345BBA3FB4476510A4BED4C78D13A'}
response = requests.get(url, headers=headers)
for stock in response.json()['OutBlock_1']:
    print(stock['ISU_CD'], stock['ISU_NM'])

088980 맥쿼리인프라
094800 맵스리얼티1
204210 모두투어리츠
096300 베트남개발1
140910 에이리츠
900140 엘브이엠씨홀딩스
099340 하나니켈1호
099350 하나니켈2호
152550 한국ANKOR유전
168490 한국패러랠


In [14]:
def get_per(stock_code):
    mkt_cap_list = []
    for year in range(2021, 2024):
        url = 'http://data-dbg.krx.co.kr/svc/sample/apis/sto/stk_bydd_trd?basDd=' + str(year) +'0102'
        headers = { 'AUTH_KEY': '74D1B99DFBF345BBA3FB4476510A4BED4C78D13A'}
        response = requests.get(url, headers=headers)
        for stock in response.json()['OutBlock_1']:
            if stock['ISU_CD'] == stock_code:
                mkt_cap_list.append(int(stock['MKT_CAP']))
    df = dart.finstate(stock_code, 2023)
    df = df[df["fs_div"] == "CFS"]
    corpfile = df[['account_nm','thstrm_amount','frmtrm_amount','bfefrmtrm_amount']].set_index('account_nm')
    profit = corpfile.loc['당기순이익'].tolist()
    for value in profit:
        try:
            clean_value = int(value.replace(',', ''))
            profit = [clean_value if x == value else x for x in profit]
        except ValueError as e:
            print(f"Error: {e}")
    profit = profit[:, :, -1]
    return [mkt_cap_list[a]/int(profit[a]) for a in range(3)]

In [ ]:
def get_profit(stock_code):
    df = dart.finstate(stock_code, 2023)
    df = df[df["fs_div"] == "CFS"]
    corpfile = df[['account_nm','thstrm_amount','frmtrm_amount','bfefrmtrm_amount']].set_index('account_nm')
    profit = corpfile.loc['당기순이익'].tolist()
    for value in profit:
        try:
            clean_value = int(value.replace(',', ''))
            profit = [clean_value if x == value else x for x in profit]
        except ValueError as e:
            print(f"Error: {e}")
    return profit[:, :, -1]

In [16]:
def find_what_we_want():
    for stock_code in df_names['종목코드']:
        try:
            roe = get_roe(stock_code)
            per = get_per(stock_code)
            profit = get_profit(stock_code)
            if roe[2] > 15 and per[2] < 50 and profit[2] > profit[1] > profit[0]:
                print(stock_code, df_names[df_names['종목코드'] == stock_code]['종목명'])
        except:
            continue